# Domain: Transportation

## Topic: Autonomous vehicles

### Team: Optimax

We are testing algorithms for Autonomous vehicles. <br>
Using carla simulator to testing in virtural enviroment.<br>
The CARLA simulator consists of a scalable client-server architecture.

In [5]:
print("Hello world")

Hello world


First need to install carla 

In [ ]:
# !pip install carla

In [6]:
import carla
import math
import random
import time
import numpy as np
# import tensorflow as tf
import cv2

Lets create a client and connect to carla local server

In [20]:
try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(5.0)
    world = client.get_world()
    print("connected with Server")
except Exception as e:
    print("An exception occurred:")
    print(e)

connected with Server


In [ ]:
print(client.get_available_maps())

In [ ]:
# world = client.load_world('Town04')

For Synchronous mode run bellow one cell

In [ ]:
# settings = world.get_settings()
# settings.synchronous_mode = True
# # settings.fixed_delta_seconds = 0.05
# world.apply_settings(settings)

For Asynchronous mode run bellow one cell

In [ ]:
# settings.synchronous_mode = False
# world.apply_settings(settings)

In [8]:
blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

Lets create some vehicles for the city and make it as someone is driving it 

In [9]:
npc_list = []
car_obj_list = [] 

# NPC for City

In [10]:
for i in range(50):
    vehicle_bp = random.choice(blueprint_library.filter('vehicle'))
    try:
        npc = world.spawn_actor(vehicle_bp, random.choice(spawn_points))
        npc_list.append(npc)
    except:
        print("one vehicle not spawn!")

one vehicle not spawn!
one vehicle not spawn!
one vehicle not spawn!
one vehicle not spawn!
one vehicle not spawn!
one vehicle not spawn!
one vehicle not spawn!


In [12]:
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [13]:
# for npc in npc_list:
#     print(npc)

# Lets start our model

Create a vehicle and locate its location

In [18]:
for a in range(50):
    try:
        car = world.spawn_actor(
            blueprint_library.find(
                random.choice([
                    'vehicle.dogde.charger_2020',
                    'vehicle.ford.mustang',
                    'vehicle.mercedes.coupe_2020',
                    'vehicle.tesla.model3'
                ])
            ),
            random.choice(spawn_points)
        )
        car_obj_list.append(car)
        print("selected")
        break
    except:
        print("Next vehicle")

selected


In [19]:
# while True:
spectator = world.get_spectator()
transform = carla.Transform(car.get_transform().transform(carla.Location(x=-5, z=2.5)), car.get_transform().rotation)
spectator.set_transform(transform)

In [21]:
for i in car_obj_list:
    print(i)

Actor(id=67, type=vehicle.mercedes.coupe_2020)
Actor(id=68, type=vehicle.tesla.model3)


In [ ]:
# Run above

In [17]:
car.set_autopilot(True)

In [ ]:
# Turn on position lights    # Position --> ON    # NONE --> OFF
car.set_light_state(carla.VehicleLightState.Position)

## Control of sensors 

In [ ]:
IMG_WIDTH = 640
IMG_HEIGHT = 480

In [ ]:
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(IMG_WIDTH))
camera_bp.set_attribute('image_size_y', str(IMG_HEIGHT))
camera_bp.set_attribute("fov", "110")

camera_init_trans = carla.Transform(carla.Location(x = 2.5, z=2.0))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = car)
car_obj_list.append(camera)

In [ ]:
def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IMG_HEIGHT, IMG_WIDTH, 4))
    i3 = i2[:, :, :3]
    cv2.imshow("", i3)
    cv2.waitKey(1)
    return i3/255.0


try:
    camera.listen(lambda data: process_img(data))
    time.sleep(10)
    camera.destroy()
except Exception as e:
    print(e)

In [ ]:
car.set_simulate_physics(True)

In [ ]:
try:
    for obj in car_obj_list:
        obj.destroy()
    print("All Objects of Model are cleaned")
except Exception as e:
    print(e)

In [ ]:
try:
    for npc in npc_list:
        npc.destroy()
    print("All NPC cleaned")
except Exception as e: 
    print(e)